In [17]:
import numpy as np

In [18]:
# Top of http://deeplearning.net/software/theano/tutorial/adding.html
a = np.asarray([[1., 2], [3, 4], [5, 6]])
b = 2.0
a * b

array([[  2.,   4.],
       [  6.,   8.],
       [ 10.,  12.]])

In [19]:
import theano.tensor as T
from theano import function
# x, y, and z are all "variables"
x = T.dscalar('x') # (d)ouble (scalar) 
y = T.dscalar('y')
z = x + y
f = function([x, y], z)
f(2, 3)

array(5.0)

In [20]:
# Theano has a concept of types that is separate from the Python type system
print(type(x))
print(x.type)
print(T.dscalar)
x.type is T.dscalar

<class 'theano.tensor.var.TensorVariable'>
TensorType(float64, scalar)
TensorType(float64, scalar)


True

In [21]:
from theano import pp
pp(z) # pretty print

'(x + y)'

In [22]:
# variable.eval() can be used instead of creating a function object
z.eval({x : 2, y : 3}) == function([x, y], z)(2, 3)

True

In [23]:
x = T.dmatrix('x') #(d)ouble (matrix)
y = T.dmatrix('y')
z = x + y
f = function([x, y], z)
print(f([[1, 2], [3, 4]], [[10, 20], [30, 40]]))
print(f([[1, 2, 3], [4, 5, 6], [7, 8, 9]], [[10, 20, 30], [40, 50, 60], [70, 80, 90]]))
try:
    print(f([[1, 2], [3, 4]], [[10, 20, 30], [40, 50, 60], [70, 80, 90]]))
except Exception:
    print("Only works on compatible matrix types.")

[[ 11.  22.]
 [ 33.  44.]]
[[ 11.  22.  33.]
 [ 44.  55.  66.]
 [ 77.  88.  99.]]
Only works on compatible matrix types.


In [24]:
# exercise at the bottom of http://deeplearning.net/software/theano/tutorial/adding.html
a = T.vector()
b = T.vector()
out = a ** 2 + b ** 2 + 2 * a * b
f = function([a, b], out)
f([1, 2, 3], [10, 20, 30])

array([  121.,   484.,  1089.])

In [25]:
f = function([a, b], a ** 2 + b ** 2 + 2 * a * b)
f([1, 2, 3], [10, 20, 30])

array([  121.,   484.,  1089.])

In [33]:
# top of http://deeplearning.net/software/theano/tutorial/examples.html
x = T.dmatrix('x')

s = 1 / (1 + T.exp(-x))
logistic = function([x], s)

s_equivalent = (1 + T.tanh(x / 2)) / 2 # equal to 1 / (1 + T.exp(-x))
logistic_equivalent = function([x], s)

logistic([[-1, 0], [1, 999]]) == logistic_equivalent([[-1, 0], [1, 999]])

array([[ True,  True],
       [ True,  True]], dtype=bool)

In [37]:
a, b = T.dmatrices('a', 'b')
diff = a - b
abs_diff = abs(diff)
diff_squared = diff ** 2
f = function([a, b], [diff, abs_diff, diff_squared])
f([[1, 1], [1, 1]], [[0, 1,], [2, 3]])

[array([[ 1.,  0.],
        [-1., -2.]]), array([[ 1.,  0.],
        [ 1.,  2.]]), array([[ 1.,  0.],
        [ 1.,  4.]])]

In [42]:
from theano import Param
x, y = T.dscalars('x', 'y')
z = x + y
f = function([x, Param(y, default=1, name='y_value')], z)
f(33) == f(33, 1) == f(33, y_value=1)

False

In [52]:
from theano import shared
state = shared(0)
inc = T.iscalar('inc')
accumulator = function([inc], state, updates=[(state, state+inc)])

In [62]:
state.set_value(0)
# Functions' updates are computed after the return value
print(accumulator(2))
print(state.get_value())
# Multiple functions can use the same shared variables
decrementor = function([inc], state, updates=[(state, state - inc)])
decrementor(5)
print(state.get_value())

0
2
-3


In [65]:
state.set_value(10)
fn_of_state = state * 2 + inc
foo = T.scalar(dtype=state.dtype)
# We can substitute a different (non-shared) value for state with the givens parameter
skip_shared = function([inc, foo], fn_of_state, givens=[(state, foo)])
skip_shared(1, 3)

array(7)

In [69]:
from theano.tensor.shared_randomstreams import RandomStreams
srng = RandomStreams(seed=234)
rv_u = srng.uniform((2,2)) # Random stream of 2 x 2 matricies, uniform distribution
rv_n = srng.normal((2,2)) # """""", normal distribution
f = function([], rv_u)
g = function([], rv_n, no_default_updates=True) # Don't update rv_n.rng
nearly_zeros = function([], rv_u + rv_u - 2 * rv_u) # random variables are drawn up to once per function execution

In [85]:
print(f() != f()) # because it gets new random numbers from rv_u each time.
print(g() == g()) # because of no_default_updates=True
nearly_zeros()

[[ True  True]
 [ True  True]]
[[ True  True]
 [ True  True]]


array([[ 0.,  0.],
       [ 0.,  0.]])

In [90]:
rng_val = rv_u.rng.get_value(borrow=True)
rng_val.seed(89234) # seed just this RandomVariable
rv_u.rng.set_value(rng_val, borrow=True) 

srng.seed(902340) # seed all RandomVariables generated from this RandomStreams object

In [96]:
print(type(rv_u.rng))
print(type(rv_u))
print(rv_u.type)
state_after_v0 = rv_u.rng.get_value().get_state() # get the state of the RandomVariable's RandomStateSharedVariable
nearly_zeros() # a function of rv_u, mutates rng state
v1 = f() # another function of rv_u, mutates rng state again
rng = rv_u.rng.get_value(borrow=True)
rng.set_state(state_after_v0) # reset state to what it was before first call
v2 = f()
v3 = f()
print(v1 == v2)
print(v1 == v3)

<class 'theano.tensor.shared_randomstreams.RandomStateSharedVariable'>
<class 'theano.tensor.var.TensorVariable'>
TensorType(float64, matrix)
[[False False]
 [False False]]
[[ True  True]
 [ True  True]]


In [66]:
rng = np.random

N = 400
feature_count = 784
learning_rate = 0.1
training_steps = 10000
# initalize a tuple for the data, D, where D[0] is a matrix of the examples (N x features) and D[1] is a list of labels (0 or 1) 
data = (rng.randn(N, feature_count), rng.randint(size=N, low=0, high=2))
#print(rng.randn(N, feature_count)) # Examples
#print(rng.randint(size=N, low=0, high=2)) # Labels

features = T.matrix('feature_matrix')
labels = T.vector('label_vector')
weights = shared(rng.randn(feature_count), name='weights') # initialize weights to random
bias = shared(0., name='bias') # initialize bias to 0
print("Initial model:")
print(weights.get_value(), bias.get_value())

p_1 = 1 / (1 + T.exp(-T.dot(features, weights) - bias)) # P(label = 1)
prediction = p_1 > 0.5
cross_entropy = -labels * T.log(p_1) - (1 - labels) * T.log(1 - p_1)
cost = cross_entropy.mean() + 0.01 * (weights ** 2).sum()
weight_gradient, bias_gradient = T.grad(cost, [weights, bias])

train = function(
    inputs=[features, labels],
    outputs=[prediction, cross_entropy],
    updates=[
        (weights, weights - learning_rate * weight_gradient),
        (bias, bias - learning_rate * bias_gradient)
    ])
predict = function(inputs=[features], outputs=prediction)

for i in range(training_steps):
    pred, error = train(data[0], data[1])

print("Final model:")
print(weights.get_value(), bias.get_value())
print("Target values for data: ", data[1])
print("Predicted values for data: ", predict(data[0]))
print("Model errors: ", data[1] - predict(data[0]))

Initial model:
[  1.24033370e+00   2.30729357e+00   2.41860984e-01  -1.02598417e+00
   6.95499761e-01   7.64626894e-01  -9.08475816e-01   1.06343074e+00
  -1.29910153e+00  -7.83751304e-01  -1.43759281e+00   4.88422651e-01
   4.30407648e-02  -9.25023807e-02  -9.04661637e-01   1.93020102e-02
  -1.90588671e+00  -1.35212098e+00   1.04821352e+00   1.46765242e-01
   2.70226386e-01  -1.27982952e+00   8.55753806e-01  -3.34431793e-01
  -3.18283149e-01   9.05399831e-01   6.66738545e-01  -9.62847757e-01
  -5.24047104e-01   5.50513183e-01  -1.40039165e-01  -9.01133533e-01
   6.63040705e-01   9.47165957e-01   1.06292958e+00   1.03087879e+00
   7.00621515e-01  -1.02680656e+00   5.04101749e-01  -5.01141360e-01
   4.13595431e-01   1.78141118e+00   1.58741770e+00  -8.27157133e-01
   1.46621258e+00  -2.90431576e-01   1.15688853e-01   3.61491520e-01
   3.11135787e-01   1.70615755e+00  -7.04369829e-01   4.34569655e-02
   6.93579020e-01   7.75199916e-01  -3.55416873e-01   9.50934520e-01
   3.72048650e-01  